来自于《统计学习方法》- AdaBoost 的例题8.1

In [1]:
import math

In [2]:
# create dataset
x = [0,1,2,3,4,5,6,7,8,9]
y = [1 for i in range(len(x))]
for i in [3,4,5,9]:
    y[i] = -1
n = len(x)
print(n)

10


In [3]:
def func_G(thred, x):
    return 1 if x <= thred else -1, -1 if x <= thred else 1    

def total_pre(func_G_list, alpha_list):
    total_pre = list()
    for pos in range(n):
        curr = 0
        for i in range(len(func_G_list)):
            curr += func_G_list[i][pos] * alpha_list[i]
        total_pre.append(1 if curr >= 0 else -1)
    return total_pre

def cal_acc(y = None, pre = None, D = None):
    assert len(y) == len(pre) and len(y) > 0
    res, n = 0, len(y)
    for i in range(n):
        if y[i] == pre[i]:
            res += D[i]
    return round(res,4)

print('test: ',cal_acc(y, [1 for _ in range(len(y))], [1/n for _ in range(len(y))]))

test:  0.6


- 1. 初始化数据权值分布 D
- 2. 对m次（第m个分类器）
    - 2.1 基于 D 权重的数据进行学习 G
    - 2.2 计算 G 分类器的误差率 e
    - 2.3 计算 G 的系数（分类器本身的权重） alpha
    - 2.4 根据误差率 e 和 分类器权重 alpha 更新数据分布 D
- 3. 构建基本分类器 G 的线性组合 

In [4]:
n = len(x)
func_G_list, alpha_list = list(), list()

for epoch in range(10):
    if epoch == 0:
        D = [1/n for _ in range(n)]
    else:
        w_list = [D[i] * math.exp(-alpha * y[i] * pre_best[i]) for i in range(n)]
        z_sum = sum(w_list)
        D = [round(w_list[i] / z_sum, 5) for i in range(n)]
    
    # 计算最低误差率时的分类器
    acc_max, acc_list = 0, list()
    for thred in range(-1, n + 1):
        pre = [func_G(thred, i)[0] for i in range(n)]
        acc = cal_acc(y, pre, D)
        pre_rev = [func_G(thred, i)[1] for i in range(n)]
        acc_rev = cal_acc(y, pre_rev, D)
        if acc > acc_max:
            thred_best = thred
            acc_max = acc
            pre_best = pre
        if acc_rev > acc_max:
            thred_best = thred
            acc_max = acc_rev
            pre_best = pre_rev
#         if epoch==2:
#             print(thred, acc_max, thred_best, pre_best)
    func_G_list.append(pre_best)    
    alpha = 1 / 2 * math.log(round(acc_max,4) / round(1 - acc_max, 4))
    alpha_list.append(alpha)
#     print(epoch, thred_best, acc_max, alpha, pre_best, D, '\n')
    total_acc = cal_acc(y, total_pre(func_G_list, alpha_list),  [1/n for _ in range(n)])
    print('epoch:', epoch, 'total_acc =', total_acc)
    if total_acc == 1:
        print('-- epoch end --')
        print("best combination") 
        for i in range(len(func_G_list)):
            print('alpha =', round(alpha_list[i],4), 'classifier:', func_G_list[i])
        break

epoch: 0 total_acc = 0.7
epoch: 1 total_acc = 0.7
epoch: 2 total_acc = 1.0
-- epoch end --
best combination
alpha = 0.4236 classifier: [1, 1, 1, -1, -1, -1, -1, -1, -1, -1]
alpha = 0.6496 classifier: [1, 1, 1, 1, 1, 1, 1, 1, 1, -1]
alpha = 0.7521 classifier: [-1, -1, -1, -1, -1, -1, 1, 1, 1, 1]


In [5]:
alpha_list

[0.42364893019360184, 0.6495990688511223, 0.75209981187596]